## 1. Import & Preprocess

In [1]:
# silence warning about shapely and pygeos interaction
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from shapely.geometry import Point
import geopandas as gpd

import folium

import os
os.chdir('../')

### 1.1 Import & Process Rain Garden Locations

In [2]:
rain = pd.read_csv("data/input/raingardens.csv")

# transform csv into geodataframe
rain = gpd.GeoDataFrame(rain, crs='epsg:4269', geometry=gpd.points_from_xy(rain.Longitude, rain.Latitude))

rain.rename(columns={'Area of Rain Garden':'garden_area', 'Address':'address', 'Latitude':'lat', 'Longitude':'lon'}, inplace=True)

# because some rows include 'width x length' in the area column, find all rows that contain an x
# and then multiply the numbers on either side of the x together and store the result
# otherwise, save the reported area as an int rather than string

rain.garden_area = ([int(area.split('x')[0]) * int(area.split('x')[1]) if 
                        ('x' in area) else int(area) for area in rain.garden_area])

rain['address'] = rain.address.str.split(",",1).str[0]
rain['address'] = rain.address.str.replace(" New Orleans", "")

print(rain.shape)
rain.sample(5)

(40, 7)


,ID,address,lat,lon,garden_area,unit of size,geometry
32,33,1314 Elysian Fields Avenue,29.971817,-90.057344,2163,sq. ft,POINT (-90.05734 29.97182)
8,9,3609 First St.,29.949624,-90.094858,2400,sq. ft,POINT (-90.09486 29.94962)
19,20,4749-61 Rosemont,30.020185,-90.000219,6832,sq. ft,POINT (-90.00022 30.02018)
37,38,4817 Dauphine St.,29.961590,-90.025803,4200,sq. ft,POINT (-90.02580 29.96159)
35,36,1901 Caffin Ave,29.970848,-90.012938,3875,sq. ft,POINT (-90.01294 29.97085)


### 1,2 Import & Process Census Block Groups

In [3]:
la_bg = gpd.read_file("data/input/la_bg_geo.shp")

la_bg = la_bg[['GEOID', 'NAME', 'geometry']].copy()

# drop invalid shapes
la_bg = la_bg.loc[la_bg.is_valid]
la_bg.reset_index(inplace=True, drop=True)

# change to standard ESPG for geospatial merging and plotting
la_bg = la_bg.to_crs(epsg=4269)

In [4]:
## bring in census data for two variables of interest
bg_vars = pd.read_csv("data/input/la_bg_vars.csv")
bg_vars.GEOID = bg_vars.GEOID.astype('str')

# rename relevant columns for ease of interpretation
bg_vars.rename(columns={
                'B02001_001_est':'total_race_pop',
                'B02001_002_est':'white_pop',
                'C17002_001_est':'total_povlvl_pop',
                'C17002_002_est':'under_povlvl_1',
                'C17002_003_est':'under_povlvl_2'
                }, inplace=True)

# calculate percent non-white as the proportion of 1 - white population / total population
bg_vars['percent_nonwhite'] = (1-round(bg_vars.white_pop / bg_vars.total_race_pop,2))*100
# if NA, means the calculation zeroed out and the population is 100% non-white
bg_vars['percent_nonwhite'].fillna(100, inplace=True)

# calculate percent under the poverty threshold as the two counts of 0-0.5 and 0.5-0.99 divided by total pop
bg_vars['percent_pov'] = round((bg_vars.under_povlvl_1 + bg_vars.under_povlvl_2) / bg_vars.total_povlvl_pop,2)*100

bg_vars.GEOID = bg_vars.GEOID.astype('str')

bg_vars = bg_vars[['GEOID', 'percent_nonwhite', 'percent_pov']].copy()

In [5]:
block_groups = la_bg.merge(bg_vars, how='inner', on='GEOID')

# reduce to relevant block groups and get rid of block group that only contains lake pontchartrain
block_groups = block_groups.query("NAME.str.contains('Orleans Parish')").copy()
block_groups = block_groups.query("GEOID!='220719900000'").copy()
block_groups = block_groups.fillna(0)

# create new columns formatted for data viz
block_groups['nonwhite_text'] = block_groups['percent_nonwhite'].astype('int').astype('str') + '%'
block_groups['pov_text'] = block_groups['percent_pov'].astype('int').astype('str') + '%'

block_groups.sample(5)

,GEOID,NAME,geometry,percent_nonwhite,percent_pov,nonwhite_text,pov_text
171,220710013014,"Block Group 4, Census Tract 13.01, Orleans Par...","POLYGON ((-90.04415 29.97415, -90.03975 29.973...",79.0,51.0,79%,51%
663,220710116002,"Block Group 2, Census Tract 116, Orleans Paris...","POLYGON ((-90.11982 29.92658, -90.11764 29.926...",12.0,10.0,12%,10%
744,220710090002,"Block Group 2, Census Tract 90, Orleans Parish...","POLYGON ((-90.08561 29.92786, -90.08099 29.930...",14.0,0.0,14%,0%
509,220710017492,"Block Group 2, Census Tract 17.49, Orleans Par...","POLYGON ((-89.93052 30.04165, -89.92857 30.041...",93.0,24.0,93%,24%
1593,220710115001,"Block Group 1, Census Tract 115, Orleans Paris...","POLYGON ((-90.12430 29.92827, -90.11982 29.926...",20.0,4.0,19%,4%


### 1.3 Import & Process Weather Stations

In [6]:
stations = pd.read_csv("data/input/stations.csv")

# transform csv into geodataframe
stations = gpd.GeoDataFrame(stations, crs='epsg:4269', geometry=gpd.points_from_xy(stations.longitude, stations.latitude))
stations = stations[['id', 'name', 'geometry']].copy()

### 1.4 Merge Rain Gardens & Weather Stations

We first conduct a spatial join between the Rain Gardens and Weather Stations to find the closest weather station to each rain garden. This relationship will be used to estimate rainfall based on the nearest weather station. It also allows for creating masks in other datasets to reduce the number of observations to those relevant to our query.

In [7]:
# spatial merge stations and rain, setting CRS to projections that work well with distance measures
# then reset to geographical CRS
garden_nearest_station = gpd.sjoin_nearest(rain.to_crs(epsg=3395), stations.to_crs(epsg=3395), distance_col="distances")
garden_nearest_station = garden_nearest_station.to_crs(epsg=4269)

### 1.5 Import & Process Precipitation Data

In [8]:
## Code for filtering precipitation data saved for reference, not needed to run after subset csv file created

# precip = pd.read_csv('data/input/precipitation.csv')

## create a mask for precipitation data, keeping only those that are closest to rain gardens
# m_p = precip.id.isin(garden_nearest_station.id)
# precip = precip[m_p].copy()
# precip.shape

# precip.to_csv("data/input/precip_nearest.csv", index=False)

## 2. Aggregate Precipitation Data

In [9]:
precip = pd.read_csv("data/input/precip_nearest.csv")

# split date into month and year columns
precip['date'] = pd.to_datetime(precip['date'])
precip['year'] = precip['date'].dt.year
precip['month'] = precip['date'].dt.month

### 2.1 Calculate Daily Average For Each Station

In [10]:
# average across every observation for each station, giving a daily average
daily_avg = precip[['id', 'value']].groupby([precip.id]).mean()
daily_avg.reset_index(inplace=True)
daily_avg.rename(columns={'value':'daily_avg'}, inplace=True)

### 2.2 Calculate Monthly Average for Each Station

In [11]:
# group by month and year for each station to get each month's total rainfall for months included in the data set
monthly_rainfall = precip.groupby([precip.id, precip.month, precip.year]).sum()
monthly_rainfall.reset_index(inplace=True)

# take each monthly sum and average them together for a station's monthly average
monthly_avg = monthly_rainfall[['id','value']].groupby(['id']).mean()
monthly_avg.reset_index(inplace=True)
monthly_avg.rename(columns={'value':'monthly_avg'}, inplace=True)

### 2.3 Calculate Annual Average for Each Station

In [12]:
# group by year for each station to get each year's total rainfall for years included in the data set
annual_rainfall = precip.groupby([precip.id, precip.year]).sum()
annual_rainfall.reset_index(inplace=True)

# take each year's sum and average them together for a station's annual average
yearly_avg = annual_rainfall[['id', 'value']].groupby(['id']).mean()
yearly_avg.reset_index(inplace=True)
yearly_avg.rename(columns={'value':'yearly_avg'}, inplace=True)

### 2.4 Convert Precipitation from Metric to Standard

In [13]:
avg_rainfall = daily_avg.merge(monthly_avg).merge(yearly_avg)

# convert from tenth of mm to 1 inch scale
avg_rainfall[['daily_avg', 'monthly_avg', 'yearly_avg']] = avg_rainfall[['daily_avg', 'monthly_avg', 'yearly_avg']].mul(0.0039701)

avg_rainfall.daily_avg = round(avg_rainfall.daily_avg,2)
avg_rainfall.monthly_avg = round(avg_rainfall.monthly_avg,0)
avg_rainfall.yearly_avg = round(avg_rainfall.yearly_avg,0)

avg_rainfall

,id,daily_avg,monthly_avg,yearly_avg
0,US1LAOR0006,0.19,5.0,47.0
1,US1LAOR0016,0.56,4.0,29.0
2,US1LAOR0019,0.26,8.0,94.0
3,USC00166659,0.18,5.0,63.0
4,USC00166671,0.17,5.0,61.0
5,USC00166672,0.17,5.0,60.0
6,USC00166675,0.16,5.0,58.0
7,USC00166678,0.17,5.0,54.0
8,USC00166679,0.17,5.0,59.0


### 2.4 Merge Rain Gardens & Precipitation

In [14]:
# merge, drop columns, rename for clarity
gardens_rainfall = garden_nearest_station.merge(avg_rainfall, how='inner', on='id')
gardens_rainfall = gardens_rainfall[['address', 'garden_area', 'id', 'daily_avg', 'monthly_avg', 'yearly_avg', 'geometry', 'lat', 'lon']].copy()
gardens_rainfall.rename(columns={'id':'station_id'}, inplace=True)

In [15]:
# use formula sq.ft space * rainfall (inches) * .623 = gallons captured
gardens_rainfall['daily_gallons'] = round(gardens_rainfall.garden_area * gardens_rainfall.daily_avg * 0.623,0)
gardens_rainfall['monthly_gallons'] = round(gardens_rainfall.garden_area * gardens_rainfall.monthly_avg * 0.623,0)
gardens_rainfall['yearly_gallons'] = round(gardens_rainfall.garden_area * gardens_rainfall.yearly_avg * 0.623,0)

gardens_rainfall.sample(5)

,address,garden_area,station_id,daily_avg,monthly_avg,yearly_avg,geometry,lat,lon,daily_gallons,monthly_gallons,yearly_gallons
16,4737 Virgilian,5590,USC00166678,0.17,5.0,54.0,POINT (-90.01675 30.01700),30.017004,-90.016750,592.0,17413.0,188059.0
7,2525 S Johnson St,3000,USC00166659,0.18,5.0,63.0,POINT (-90.09440 29.94865),29.948648,-90.094404,336.0,9345.0,117747.0
6,3210 Delachaise St,5911,USC00166671,0.17,5.0,61.0,POINT (-90.09906 29.94455),29.944546,-90.099063,626.0,18413.0,224636.0
19,2609 Onzaga St.,1568,US1LAOR0016,0.56,4.0,29.0,POINT (-90.07476 29.98143),29.981429,-90.074764,547.0,3907.0,28329.0
15,4749-61 Rosemont,6832,USC00166678,0.17,5.0,54.0,POINT (-90.00022 30.02018),30.020185,-90.000219,724.0,21282.0,229842.0


### 2.5 Create Crosswalk Between Census and Gardens

In [16]:
gardens_bg = gardens_rainfall.sjoin(block_groups, predicate='within', how='inner')
gardens_bg = gardens_bg[['address', 'GEOID']].copy()

### 2.6 Export Datasets

In [17]:
# for both final datasets, drop the geometry for lighter files when plotting variables of interest in ggplot

gardens_rainfall_out = gardens_rainfall.drop('geometry', axis=1)
gardens_rainfall_out.to_csv("data/output/gardens_rainfall.csv", index=False)

block_groups_out = block_groups.drop('geometry', axis=1)
block_groups_out.to_csv("data/output/block_groups.csv", index=False)

gardens_bg.to_csv("data/output/gardens_block_groups.csv", index=False)

## 3. Data Vizualizations

### 3.1 Interactive Map

In [18]:
# Initialize the basemap for contextual location
m = folium.Map(location=[29.995, -90.05], zoom_start=12, tiles=None)

# store the two choropleths and features into groups for layering
fg1 = folium.FeatureGroup(name='Percent Non-White',overlay=False).add_to(m)
fg2 = folium.FeatureGroup(name='Percent Below Povery Line',overlay=False).add_to(m)

# Create choropleth map
ch1 = folium.Choropleth(
            name='Percent Non-White',
            geo_data=block_groups,
            data=block_groups,
            columns=['GEOID', 'percent_nonwhite'],
            key_on='feature.properties.GEOID',
            bins=9,
            fill_color='Purples',
            fill_opacity=0.6, 
            line_opacity=0.3,
            highlight=True, 
            line_color='black').geojson.add_to(fg1)

# Create the tooltip for additional details
ch1.add_child(
    folium.features.GeoJsonPopup(['GEOID', 'nonwhite_text', 'pov_text'], labels=True, 
                                   aliases=['GEOID', 'Non-White Population', 'Below Povery Line'])
)

# Create choropleth map
ch2 = folium.Choropleth(
            name='Percent Below Povery Line',
            overlay=False,
            geo_data=block_groups, 
            data=block_groups,
            columns=['GEOID', 'percent_pov'],
            key_on='feature.properties.GEOID',
            fill_color='YlOrRd',
            fill_opacity=0.6, 
            line_opacity=0.3,
            highlight=True,
            line_color='black').geojson.add_to(fg2)

# Create the on-click popup for additional details for each block group
ch2.add_child(
    folium.features.GeoJsonPopup(['GEOID', 'nonwhite_text', 'pov_text'], labels=True, 
                                   aliases=['GEOID', 'Non-White Population', 'Below Povery Line'])
)

# add rain gardens as points with hover tooltips including formatted text
for i in range(0,len(gardens_rainfall)):
   folium.Marker(
      location=[gardens_rainfall.iloc[i]['lat'], gardens_rainfall.iloc[i]['lon']],
      tooltip=( '<b>Address:</b> ' + str(gardens_rainfall.at[i, 'address'])
               + '<br>'
               + '<br>' + '<b>Avg. Monthly Rainfall:</b> ' + str(int(gardens_rainfall.at[i, 'monthly_avg'])) + " inches" 
               + '<br>' + '<b>Avg. Monthly Rainfall Captured:</b> ' + str(int(gardens_rainfall.at[i, 'monthly_gallons'])) + " gallons"
               + "<br>"
               + '<br>' + '<b>Avg. Annual Rainfall:</b> ' + str(int(gardens_rainfall.at[i, 'yearly_avg'])) + " inches" 
               + '<br>' + '<b>Avg. Annual Rainfall Captured:</b> ' + str(int(gardens_rainfall.at[i, 'yearly_gallons'])) + " gallons"
               ),
   ).add_to(m)

# add basemap overlay
folium.TileLayer('cartodbpositron',overlay=True,name="Basemap").add_to(m)

# add layer controls to map, display by default
folium.LayerControl(collapsed=False).add_to(m)

#display the map layers
m

In [19]:
m.save("raingarden_map.html")